In [1]:
import requests
from bs4 import BeautifulSoup
 
import pandas as pd
import re
from datetime import datetime
import time
 
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from io import BytesIO
# from openpyxl import Workbook
from openpyxl.drawing.image import Image

In [2]:
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

In [3]:
chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

### 1. Image Crawler ver 0.1

In [27]:
# smartstore test

import urllib.request
import os

path = 'C:/Users/shopperhouse/Downloads/'
data_folder = 'C:/Users/shopperhouse/Downloads/test_ver'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]

count = 0
# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 리뷰를 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 1. 셀레니움으로 html 가져오기
    html_source = driver.page_source
    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    # se-module-image-link __se_image_link __se_link
    # 4. 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 리뷰 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        # reivew_image_tag = driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > ul > li:nth-child(1) > div > div > div > div._3-1uaKhzq4 > div > div._3Bbv1ae9fg > span > img')
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'C:/Users/shopperhouse/Downloads/test_ver/{count}.jpg')
            count += 1

print('done')
driver.quit()

Review Image Link: https://shop-phinf.pstatic.net/20240401_203/1711940739795TufPU_JPEG/%EB%8F%84%EC%B0%A9%ED%99%95%EC%9C%A8.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20230721_63/1689910982371AThsI_JPEG/%EC%8A%A4%EC%B9%B4%EC%9D%B4%ED%8A%B8%EB%A6%BD.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240417_286/17133228945954lvNf_PNG/%EB%B0%B0%EC%86%A1%EC%95%88%EB%82%B4-%EC%88%98%EC%A0%95-%ED%86%B5%EC%A7%84.png?type=w860
done


In [29]:
# brandstore test

import urllib.request
import os

path = 'C:/Users/shopperhouse/Downloads/'
data_folder = 'C:/Users/shopperhouse/Downloads/test_ver'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3'
    ]

count = 0
# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 리뷰를 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 1. 셀레니움으로 html 가져오기
    html_source = driver.page_source
    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    # se-module-image-link __se_image_link __se_link
    # 4. 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 리뷰 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        # reivew_image_tag = driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > ul > li:nth-child(1) > div > div > div > div._3-1uaKhzq4 > div > div._3Bbv1ae9fg > span > img')
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'C:/Users/shopperhouse/Downloads/test_ver/{count}.jpg')
            count += 1

print('done')
driver.quit()

Review Image Link: https://shop-phinf.pstatic.net/20240610_262/1717984740336lHk5b_JPEG/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%EB%B0%B0%EC%86%A1%EC%95%88%EB%82%B4_%EA%B8%B0%EB%B3%B8_15%EC%8B%9C(%EB%A6%AC%EC%82%AC%EC%9D%B4%EC%A7%95).jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240702_192/17199089684425kBNz_JPEG/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_7%EC%9B%94-%ED%94%84%EB%A1%9C%EB%AA%A8%EC%85%98_web.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240625_201/17193047145950QThY_JPEG/7%EC%9B%94_%EC%BF%A0%ED%8F%B0PSD.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240503_216/1714717213463aeVAF_GIF/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%ED%99%80%EB%A1%9C%EA%B7%B8%EB%9E%A8.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_267/1653271247367BRAlq_GIF/yosul_100-1.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_112/1653271247487oQIzT_GIF/yosul_100-2.gif?type=wg860
Review Image Link: https://shop

### 2. jpg to pdf

In [28]:

import os
from PIL import Image
from fpdf import FPDF

# Define the path to your folder containing images
data_folder = "C:/Users/shopperhouse/Downloads/test_ver"

# Define the PDF dimensions (A4 size in mm)
pdf_width = 210
pdf_height = 297

# Create a PDF object
pdf = FPDF()

# Iterate through the image files in the folder
for image_file in os.listdir(data_folder):
    if image_file.endswith(".jpg"):
        image_path = os.path.join(data_folder, image_file)
        print(image_path)
        
        # Open the image file
        image = Image.open(image_path)
        
        # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
        width_mm = image.width * 25.4 / 72
        height_mm = image.height * 25.4 / 72
        
        # Calculate the scaling factor to fit the image within the PDF page
        scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)
        
        # Calculate new dimensions while maintaining the aspect ratio
        new_width_mm = width_mm * scaling_factor
        new_height_mm = height_mm * scaling_factor
        
        # Calculate the position to center the image on the page
        x_position = (pdf_width - new_width_mm) / 2
        y_position = (pdf_height - new_height_mm) / 2
        
        # Resize the image maintaining aspect ratio
        new_width = int(image.width * scaling_factor)
        new_height = int(image.height * scaling_factor)
        resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Save the resized image to a temporary path
        temp_image_path = os.path.join(data_folder, "temp.jpg")
        resized_image.save(temp_image_path)
        
        # Add a new page to the PDF
        pdf.add_page()
        
        # Insert the image into the PDF at the calculated position
        pdf.image(image_path, x=x_position, y=y_position, w=new_width_mm, h=new_height_mm)

# Define the output PDF file path
pdf_file_path = os.path.join(data_folder, "images.pdf")

# Save the PDF file
pdf.output(pdf_file_path)
print(f"PDF saved at {pdf_file_path}")


C:/Users/shopperhouse/Downloads/test_ver\0.jpg
C:/Users/shopperhouse/Downloads/test_ver\1.jpg
C:/Users/shopperhouse/Downloads/test_ver\2.jpg
PDF saved at C:/Users/shopperhouse/Downloads/test_ver\images.pdf


### 3. save data to s3

In [ ]:
def save_image_data(ecommerce, crawling_type, category, pdf_file_path, ctg_fullname, file_format='pdf', bucket_name='ailee-crawling-data'):
    now = datetime.now()
    file_name = f'{ctg_fullname}_{now.strftime("%Y%m%d")}.{file_format}'
    s3_folder_path = f'{ecommerce}-detailpage/{crawling_type}/{category}/'
    s3_key = s3_folder_path + file_name

    # Initialize S3 client
    dotenv_path = Path("C:/Users/shopperhouse/Code/GEN_Ailee/.env") # your path
    load_dotenv(dotenv_path=dotenv_path)
    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY_ID = os.getenv("AWS_SECRET_ACCESS_KEY_ID")
    s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY_ID)

    # Upload the data to S3
    try:
        s3.upload_file(pdf_file_path, bucket_name, s3_key)
        print(f"PDF successfully uploaded to s3://{bucket_name}/{s3_key}")
    except Exception as e:
        print(f"Error uploading PDF to S3: {e}")

In [ ]:
save_image_data('naver', 'image', 'travel-adapter')

Error uploading PDF to S3: name 'pdf_file_path' is not defined


In [5]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%Y-%m-%d")

In [7]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
 
import pandas as pd
import re
from datetime import datetime, timedelta
import time

from io import BytesIO
from openpyxl.drawing.image import Image

from upload_data import *

options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d")

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3',
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]

# url이 df로 들어왔을 때
# urls = df.urls.to_list()

count = 0
# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 브랜드명 및 제품명 수집
    try: 
        brand = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        product = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(3) > td:nth-child(2)').text        

    # brand_name.append(brand)
    # product_name.append(product)
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")

    folder_name = f"{brand}_{product}"
    folder_path = os.path.join('C:/Users/shopperhouse/Downloads/', folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    # se-module-image-link __se_image_link __se_link
    
    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        # reivew_image_tag = driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > ul > li:nth-child(1) > div > div > div > div._3-1uaKhzq4 > div > div._3Bbv1ae9fg > span > img')
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'{folder_path}/{product}_{current_time}_{count}.jpg')
            count += 1

print('done')
driver.quit()

# ========================================== jpg to pdf =============================================
# Define the PDF dimensions (A4 size in mm)
ppdf_width = 210
pdf_height = 297

# Create a PDF object
pdf = FPDF()

# Iterate through the image files in the folder
for image_file in os.listdir(folder_path):
    if image_file.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(folder_path, image_file)
        print(image_path)
        
        # Open the image file
        image = Image.open(image_path)

        # 이미지 rgb모드로 변환 convert image to rgb
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
        width_mm = image.width * 25.4 / 72
        height_mm = image.height * 25.4 / 72
        
        # Calculate the scaling factor to fit the image within the PDF page
        scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)
        
        # Calculate new dimensions while maintaining the aspect ratio
        new_width_mm = width_mm * scaling_factor
        new_height_mm = height_mm * scaling_factor
        
        # Calculate the position to center the image on the page
        x_position = (pdf_width - new_width_mm) / 2
        y_position = (pdf_height - new_height_mm) / 2
        
        # Resize the image maintaining aspect ratio
        new_width = int(image.width * scaling_factor)
        new_height = int(image.height * scaling_factor)
        resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Save the resized image to a temporary path
        temp_image_path = os.path.join(folder_path, "temp.jpg") # 병합된 파일 임의로 저장
        resized_image.save(temp_image_path, format='JPEG')

        # Optionally remove the temporary image file
        # if temp_image_path != image_path:
        #     os.remove(temp_image_path)
        
        # Add a new page to the PDF
        pdf.add_page()
        
        # Insert the image into the PDF at the calculated position
        pdf.image(image_path, x=x_position, y=y_position, w=new_width_mm, h=new_height_mm)

        # Define the output PDF file path
        pdf_file_path = os.path.join(folder_path, "fimages.pdf")

        # Save the PDF file
        pdf.output(pdf_file_path)
        print(f"PDF saved at {pdf_file_path}")

        save_image_data('naver', 'image', 'travel-adapter')


Review Image Link: https://shop-phinf.pstatic.net/20240610_262/1717984740336lHk5b_JPEG/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%EB%B0%B0%EC%86%A1%EC%95%88%EB%82%B4_%EA%B8%B0%EB%B3%B8_15%EC%8B%9C(%EB%A6%AC%EC%82%AC%EC%9D%B4%EC%A7%95).jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240503_216/1714717213463aeVAF_GIF/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%ED%99%80%EB%A1%9C%EA%B7%B8%EB%9E%A8.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_267/1653271247367BRAlq_GIF/yosul_100-1.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_112/1653271247487oQIzT_GIF/yosul_100-2.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20221201_117/1669858739081SX1Dz_GIF/black_exp.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20231012_79/1697081238259sCaxE_GIF/04.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20240426_227/1714110774146lrA7o_JPEG/pad_all_01.jpg?type=w860
Review Image Link: https://sh

AttributeError: type object 'Image' has no attribute 'open'

### 4. 합쳐서 수정

1. 데이터셋 저장 시 brand, product name, current time으로 구분
2. s3로 바로 저장
3. url이 칼럼으로 들어올 때
4. 된다면 보안 로직 추가
-> user_agent 넣으니까 오히려 크롤링 바로 막힘

In [ ]:
# # log에 남아있던 user_agent_list
# user_agent_list = [
#   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
#   'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
#   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
#   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
#   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
# ]

In [9]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import pandas as pd
import re
from datetime import datetime
import time

import os
from io import BytesIO
from openpyxl.drawing.image import Image
import PIL.Image
from PIL import Image
from fpdf import FPDF

from upload_data import *

#-------web driver setting-----------
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3',
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]

folder_paths = []

# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 브랜드명 및 제품명 수집
    try: 
        brand = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        product = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(3) > td:nth-child(2)').text
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")
        continue

    folder_name = f"{brand}_{product}"
    folder_path = os.path.join('C:/Users/shopperhouse/Downloads/', folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)
    folder_paths.append((folder_name, folder_path))

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    count = 0
    
    # 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    
    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'{folder_path}/{product}_{count}.jpg')
            count += 1

print('done')
driver.quit()

# PDF 변환 및 S3 업로드
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

    # S3에 업로드
    ecommerce = 'naver'
    crawling_type = 'image'
    category = '전기용품'
    file_type = 'pdf'
    ctg_fullname = folder_name
    save_image_pdf_data(ecommerce, crawling_type, category, file_type, pdf_output_path, ctg_fullname)


In [4]:
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

PDF saved to C:/Users/shopperhouse/Downloads/페츠모아_페츠모아 냄새잡는 요술배변패드 100장\페츠모아_페츠모아 냄새잡는 요술배변패드 100장.pdf
PDF saved to C:/Users/shopperhouse/Downloads/스카이트립_스카이트립 멀티플러그\스카이트립_스카이트립 멀티플러그.pdf


In [ ]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import pandas as pd
import re
from datetime import datetime
import time

import os
from io import BytesIO
from openpyxl.drawing.image import Image
import PIL.Image
from PIL import Image
from fpdf import FPDF

from upload_data import *

#-------web driver setting-----------
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/Users/tips_workstation2/Desktop/new_crawler/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3',
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]


folder_paths = []

exception_urls = []

# URL에서 데이터 수집
for url in urls:
    driver.get(url)
    actual_url = driver.current_url
    print(actual_url)

    if not (actual_url.startswith('https://brand') or actual_url.startswith('https://smartstore')):
        exception_urls.append(actual_url)
        continue

    # 브랜드명 및 제품명 수집 -> 해당 부분 수정 -> 어떤 element를 뽑을지?
    try:
        try:
            brand_element = driver.find_element(By.CSS_SELECTOR, '#pc-storeNameWidget > div > div > a > img')
            brand = brand_element.get_attribute('alt')
        except Exception as e1:
            try:
                brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
                brand = brand_element.get_attribute('alt')
            except Exception as e2:
                try:
                    brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-storeNameWidget > div > div > a > span")
                    brand = brand_element.text
                except Exception as e3:
                    try:
                        brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > span")
                        brand = brand_element.text
                    except Exception as e4:
                        print(f"브랜드명 수집 실패 : {e4}")
                        exception_urls.append(url)
                        continue

        product = driver.find_element(By.CSS_SELECTOR, '#content > div > div._2-I30XS1lA > div._2QCa6wHHPy > fieldset > div._3k440DUKzy > div._1eddO7u4UC > h3').text
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")
        exception_urls.append(url)
        continue

    sanitized_product = product.replace("/","_").replace(" ","_")
    folder_name = sanitized_product
    # folder_name = f"{brand}_{product}"
    folder_path = os.path.join("C:/Users/tips_workstation2/Desktop/C_Image/", folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)
    folder_paths.append((folder_name, folder_path))

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")


    count = 0

    try:
        # 상세정보 버튼
        driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
        time.sleep(3)

        # 상세정보 펼쳐보기 버튼
        driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
        time.sleep(3)

    except Exception as e:
        print(f"상세정보 버튼 클릭 실패: {e}")
        exception_urls.append(url)
        continue


    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 상세정보 전체
    images = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'})

    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for image in range(len(images)):
        # 4-4. 이미지 수집
        image_tag = images[image].findAll('img', {'class': 'se-image-resource'})
        if image_tag:
            image_link = image_tag[0]['data-src']
            print(f'Review Image Link: {image_link}')  # 이미지 링크 출력
            image_link = image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            local_img_path = os.path.join(folder_path, f'{count}.jpg')
            print(local_img_path)
            urllib.request.urlretrieve(image_link, local_img_path)
            count += 1

            # # s3에도 업로드
            # ecommerce = 'naver'
            # crawling_type = 'image'
            # category = '전기용품'         # 여기부터 변경
            # search_type = 'keyword'
            # file_type = 'jpg'
            # jpg_fullname = f'{folder_name}/{count}'
            # jpg_file_path = local_img_path

            # save_image_jpg_data(ecommerce, crawling_type, category, search_type, file_type, jpg_file_path, jpg_fullname)

print('done')
driver.quit()

In [8]:
local_img_path

'C:/Users/shopperhouse/Downloads/여행_멀티_플러그_해외_여행용_어댑터_일본_멀티탭_콘센트_멀티_어댑터_유럽_중국\\2.jpg'

In [7]:
jpg_fullname

'여행_멀티_플러그_해외_여행용_어댑터_일본_멀티탭_콘센트_멀티_어댑터_유럽_중국/3'

In [6]:
jpg_file_path

'C:/Users/shopperhouse/Downloads/여행_멀티_플러그_해외_여행용_어댑터_일본_멀티탭_콘센트_멀티_어댑터_유럽_중국\\2.jpg'

In [ ]:
import requests
from bs4 import BeautifulSoup
 
import pandas as pd
import re
from datetime import datetime
import time
 
# selenium import
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from io import BytesIO
# from openpyxl import Workbook
from openpyxl.drawing.image import Image

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

In [ ]:
chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

### 1. Image Crawler ver 0.1

In [ ]:
# smartstore test

import urllib.request
import os

path = 'C:/Users/shopperhouse/Downloads/'
data_folder = 'C:/Users/shopperhouse/Downloads/test_ver'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]

count = 0
# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 리뷰를 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 1. 셀레니움으로 html 가져오기
    html_source = driver.page_source
    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    # se-module-image-link __se_image_link __se_link
    # 4. 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 리뷰 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        # reivew_image_tag = driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > ul > li:nth-child(1) > div > div > div > div._3-1uaKhzq4 > div > div._3Bbv1ae9fg > span > img')
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'C:/Users/shopperhouse/Downloads/test_ver/{count}.jpg')
            count += 1

print('done')
driver.quit()

Review Image Link: https://shop-phinf.pstatic.net/20240401_203/1711940739795TufPU_JPEG/%EB%8F%84%EC%B0%A9%ED%99%95%EC%9C%A8.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20230721_63/1689910982371AThsI_JPEG/%EC%8A%A4%EC%B9%B4%EC%9D%B4%ED%8A%B8%EB%A6%BD.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240417_286/17133228945954lvNf_PNG/%EB%B0%B0%EC%86%A1%EC%95%88%EB%82%B4-%EC%88%98%EC%A0%95-%ED%86%B5%EC%A7%84.png?type=w860
done


In [ ]:
# brandstore test

import urllib.request
import os

path = 'C:/Users/shopperhouse/Downloads/'
data_folder = 'C:/Users/shopperhouse/Downloads/test_ver'
if not os.path.exists(data_folder):
    os.makedirs(data_folder)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3'
    ]

count = 0
# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 리뷰를 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 1. 셀레니움으로 html 가져오기
    html_source = driver.page_source
    # 2. bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 3. 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    # se-module-image-link __se_image_link __se_link
    # 4. 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 리뷰 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        # reivew_image_tag = driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > ul > li:nth-child(1) > div > div > div > div._3-1uaKhzq4 > div > div._3Bbv1ae9fg > span > img')
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'C:/Users/shopperhouse/Downloads/test_ver/{count}.jpg')
            count += 1

print('done')
driver.quit()

Review Image Link: https://shop-phinf.pstatic.net/20240610_262/1717984740336lHk5b_JPEG/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%EB%B0%B0%EC%86%A1%EC%95%88%EB%82%B4_%EA%B8%B0%EB%B3%B8_15%EC%8B%9C(%EB%A6%AC%EC%82%AC%EC%9D%B4%EC%A7%95).jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240702_192/17199089684425kBNz_JPEG/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_7%EC%9B%94-%ED%94%84%EB%A1%9C%EB%AA%A8%EC%85%98_web.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240625_201/17193047145950QThY_JPEG/7%EC%9B%94_%EC%BF%A0%ED%8F%B0PSD.jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240503_216/1714717213463aeVAF_GIF/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%ED%99%80%EB%A1%9C%EA%B7%B8%EB%9E%A8.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_267/1653271247367BRAlq_GIF/yosul_100-1.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_112/1653271247487oQIzT_GIF/yosul_100-2.gif?type=wg860
Review Image Link: https://shop

### 2. jpg to pdf

In [ ]:

import os
from PIL import Image
from fpdf import FPDF

# Define the path to your folder containing images
data_folder = "C:/Users/shopperhouse/Downloads/test_ver"

# Define the PDF dimensions (A4 size in mm)
pdf_width = 210
pdf_height = 297

# Create a PDF object
pdf = FPDF()

# Iterate through the image files in the folder
for image_file in os.listdir(data_folder):
    if image_file.endswith(".jpg"):
        image_path = os.path.join(data_folder, image_file)
        print(image_path)
        
        # Open the image file
        image = Image.open(image_path)
        
        # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
        width_mm = image.width * 25.4 / 72
        height_mm = image.height * 25.4 / 72
        
        # Calculate the scaling factor to fit the image within the PDF page
        scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)
        
        # Calculate new dimensions while maintaining the aspect ratio
        new_width_mm = width_mm * scaling_factor
        new_height_mm = height_mm * scaling_factor
        
        # Calculate the position to center the image on the page
        x_position = (pdf_width - new_width_mm) / 2
        y_position = (pdf_height - new_height_mm) / 2
        
        # Resize the image maintaining aspect ratio
        new_width = int(image.width * scaling_factor)
        new_height = int(image.height * scaling_factor)
        resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Save the resized image to a temporary path
        temp_image_path = os.path.join(data_folder, "temp.jpg")
        resized_image.save(temp_image_path)
        
        # Add a new page to the PDF
        pdf.add_page()
        
        # Insert the image into the PDF at the calculated position
        pdf.image(image_path, x=x_position, y=y_position, w=new_width_mm, h=new_height_mm)

# Define the output PDF file path
pdf_file_path = os.path.join(data_folder, "images.pdf")

# Save the PDF file
pdf.output(pdf_file_path)
print(f"PDF saved at {pdf_file_path}")


C:/Users/shopperhouse/Downloads/test_ver\0.jpg
C:/Users/shopperhouse/Downloads/test_ver\1.jpg
C:/Users/shopperhouse/Downloads/test_ver\2.jpg
PDF saved at C:/Users/shopperhouse/Downloads/test_ver\images.pdf


### 3. save data to s3

In [ ]:
def save_image_data(ecommerce, crawling_type, category, pdf_file_path, ctg_fullname, file_format='pdf', bucket_name='ailee-crawling-data'):
    now = datetime.now()
    file_name = f'{ctg_fullname}_{now.strftime("%Y%m%d")}.{file_format}'
    s3_folder_path = f'{ecommerce}-detailpage/{crawling_type}/{category}/'
    s3_key = s3_folder_path + file_name

    # Initialize S3 client
    dotenv_path = Path("C:/Users/shopperhouse/Code/GEN_Ailee/.env") # your path
    load_dotenv(dotenv_path=dotenv_path)
    AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
    AWS_SECRET_ACCESS_KEY_ID = os.getenv("AWS_SECRET_ACCESS_KEY_ID")
    s3 = boto3.client('s3', aws_access_key_id=AWS_ACCESS_KEY_ID, aws_secret_access_key=AWS_SECRET_ACCESS_KEY_ID)

    # Upload the data to S3
    try:
        s3.upload_file(pdf_file_path, bucket_name, s3_key)
        print(f"PDF successfully uploaded to s3://{bucket_name}/{s3_key}")
    except Exception as e:
        print(f"Error uploading PDF to S3: {e}")

In [ ]:
save_image_data('naver', 'image', 'travel-adapter')

Error uploading PDF to S3: name 'pdf_file_path' is not defined


In [ ]:
from datetime import datetime
now = datetime.now()
current_time = now.strftime("%Y-%m-%d")

In [ ]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
 
import pandas as pd
import re
from datetime import datetime, timedelta
import time

from io import BytesIO
from openpyxl.drawing.image import Image

from upload_data import *

options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d")

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3',
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]

# url이 df로 들어왔을 때
# urls = df.urls.to_list()

count = 0
# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 브랜드명 및 제품명 수집
    try: 
        brand = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        product = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(3) > td:nth-child(2)').text        

    # brand_name.append(brand)
    # product_name.append(product)
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")

    folder_name = f"{brand}_{product}"
    folder_path = os.path.join('C:/Users/shopperhouse/Downloads/', folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    # se-module-image-link __se_image_link __se_link
    
    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        # reivew_image_tag = driver.find_element(By.CSS_SELECTOR, '#REVIEW > div > div._2LvIMaBiIO > div._2g7PKvqCKe > ul > li:nth-child(1) > div > div > div > div._3-1uaKhzq4 > div > div._3Bbv1ae9fg > span > img')
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'{folder_path}/{product}_{current_time}_{count}.jpg')
            count += 1

print('done')
driver.quit()

# ========================================== jpg to pdf =============================================
# Define the PDF dimensions (A4 size in mm)
ppdf_width = 210
pdf_height = 297

# Create a PDF object
pdf = FPDF()

# Iterate through the image files in the folder
for image_file in os.listdir(folder_path):
    if image_file.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(folder_path, image_file)
        print(image_path)
        
        # Open the image file
        image = Image.open(image_path)

        # 이미지 rgb모드로 변환 convert image to rgb
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
        width_mm = image.width * 25.4 / 72
        height_mm = image.height * 25.4 / 72
        
        # Calculate the scaling factor to fit the image within the PDF page
        scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)
        
        # Calculate new dimensions while maintaining the aspect ratio
        new_width_mm = width_mm * scaling_factor
        new_height_mm = height_mm * scaling_factor
        
        # Calculate the position to center the image on the page
        x_position = (pdf_width - new_width_mm) / 2
        y_position = (pdf_height - new_height_mm) / 2
        
        # Resize the image maintaining aspect ratio
        new_width = int(image.width * scaling_factor)
        new_height = int(image.height * scaling_factor)
        resized_image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)
        
        # Save the resized image to a temporary path
        temp_image_path = os.path.join(folder_path, "temp.jpg") # 병합된 파일 임의로 저장
        resized_image.save(temp_image_path, format='JPEG')

        # Optionally remove the temporary image file
        # if temp_image_path != image_path:
        #     os.remove(temp_image_path)
        
        # Add a new page to the PDF
        pdf.add_page()
        
        # Insert the image into the PDF at the calculated position
        pdf.image(image_path, x=x_position, y=y_position, w=new_width_mm, h=new_height_mm)

        # Define the output PDF file path
        pdf_file_path = os.path.join(folder_path, "fimages.pdf")

        # Save the PDF file
        pdf.output(pdf_file_path)
        print(f"PDF saved at {pdf_file_path}")

        save_image_data('naver', 'image', 'travel-adapter')


Review Image Link: https://shop-phinf.pstatic.net/20240610_262/1717984740336lHk5b_JPEG/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%EB%B0%B0%EC%86%A1%EC%95%88%EB%82%B4_%EA%B8%B0%EB%B3%B8_15%EC%8B%9C(%EB%A6%AC%EC%82%AC%EC%9D%B4%EC%A7%95).jpg?type=w860
Review Image Link: https://shop-phinf.pstatic.net/20240503_216/1714717213463aeVAF_GIF/%ED%8E%98%EC%B8%A0%EB%AA%A8%EC%95%84_%ED%99%80%EB%A1%9C%EA%B7%B8%EB%9E%A8.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_267/1653271247367BRAlq_GIF/yosul_100-1.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20220523_112/1653271247487oQIzT_GIF/yosul_100-2.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20221201_117/1669858739081SX1Dz_GIF/black_exp.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20231012_79/1697081238259sCaxE_GIF/04.gif?type=wg860
Review Image Link: https://shop-phinf.pstatic.net/20240426_227/1714110774146lrA7o_JPEG/pad_all_01.jpg?type=w860
Review Image Link: https://sh

AttributeError: type object 'Image' has no attribute 'open'

### 4. 합쳐서 수정

1. 데이터셋 저장 시 brand, product name, current time으로 구분
2. s3로 바로 저장
3. url이 칼럼으로 들어올 때
4. 된다면 보안 로직 추가
-> user_agent 넣으니까 오히려 크롤링 바로 막힘

In [ ]:
# # log에 남아있던 user_agent_list
# user_agent_list = [
#   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
#   'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
#   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
#   'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0',
#   'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
# ]

In [1]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import pandas as pd
import re
from datetime import datetime
import time

import os
from io import BytesIO
from openpyxl.drawing.image import Image
import PIL.Image
from PIL import Image
from fpdf import FPDF

from upload_data import *

#-------web driver setting-----------
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
urls = [
    'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3',
    'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
]

folder_paths = []

# URL에서 데이터 수집
for url in urls:
    driver.get(url)

    # 브랜드명 및 제품명 수집
    try: 
        brand = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(2) > td:nth-child(4)').text
        product = driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div.attribute_wrapper > div > div._2E4i2Scsp4._copyable > table > tbody > tr:nth-child(3) > td:nth-child(2)').text
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")
        continue

    folder_name = f"{brand}_{product}"
    folder_path = os.path.join('C:/Users/shopperhouse/Downloads/', folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)
    folder_paths.append((folder_name, folder_path))

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")

    # 상세정보 버튼
    driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
    time.sleep(3)

    # 상세정보 펼쳐보기 버튼
    driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
    time.sleep(3)

    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    count = 0
    
    # 상세정보 전체
    reviews = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'}) 
    
    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for review in range(len(reviews)):
        # 4-4. 이미지 수집
        review_image_tag = reviews[review].findAll('img', {'class': 'se-image-resource'})
        if review_image_tag:
            review_image_link = review_image_tag[0]['data-src']
            print(f'Review Image Link: {review_image_link}')  # 이미지 링크 출력
            review_image_link = review_image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            urllib.request.urlretrieve(review_image_link, f'{folder_path}/{product}_{count}.jpg')
            count += 1

print('done')
driver.quit()

# PDF 변환 및 S3 업로드
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

    # S3에 업로드
    ecommerce = 'naver'
    crawling_type = 'image'
    category = '전기용품'
    file_type = 'pdf'
    ctg_fullname = folder_name
    save_image_pdf_data(ecommerce, crawling_type, category, file_type, pdf_output_path, ctg_fullname)


KeyboardInterrupt: 

In [ ]:
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

PDF saved to C:/Users/shopperhouse/Downloads/페츠모아_페츠모아 냄새잡는 요술배변패드 100장\페츠모아_페츠모아 냄새잡는 요술배변패드 100장.pdf
PDF saved to C:/Users/shopperhouse/Downloads/스카이트립_스카이트립 멀티플러그\스카이트립_스카이트립 멀티플러그.pdf


### 5. 최종

In [8]:
import requests
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import os
import pandas as pd
import re
from datetime import datetime
import time

import os
from io import BytesIO
from openpyxl.drawing.image import Image
import PIL.Image
from PIL import Image
from fpdf import FPDF

from upload_data import *

#-------web driver setting-----------
options = webdriver.ChromeOptions()
options.add_argument("window-size=1920x1080") # 화면크기(전체화면)
options.add_argument("disable-gpu")
options.add_argument("disable-infobars")
options.add_argument("--disable-extensions")
options.add_argument('--no-sandbox')
# options.add_argument('--headless')  # 브라우저를 숨기는 옵션 추가

chrome_driver_path = r'C:/Users/tips_workstation2/Desktop/new_crawler/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

In [4]:
meta = pd.read_parquet("C:/Users/shopperhouse/Downloads/meta_여행용 어댑터_parquet")
meta

,product_names,product_urls,product_prices,prodict_sellers,product_detail,product_marketings
0,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,https://cr.shopping.naver.com/adcr.nhn?x=dVYa9...,"2,000원",스카이트립,"종류 : 콘센트형, 전원스위치 : 통합스위치, 코드수 : 1구",
1,감비앤코 여행용 멀티 어댑터 해외여행 멀티탭 돼지코 전세계 멀충 PD C타입,https://cr.shopping.naver.com/adcr.nhn?x=cM9sa...,"32,000원",감비앤코,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 1구, 특징 : 과부하차단...",출시기념 특가 행사와 파우치 증정 이벤트
2,지티스 여행용 멀티 어댑터 해외여행 C타입 초고속 전세계 110V 멀티탭,https://cr.shopping.naver.com/adcr.nhn?x=Kve9j...,"31,800원",지티스몰,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 1구, 특징 : 과부하차단...",알림받기 즉시할인 및 포토영상 리뷰 작성 시 즉시! 1000포인트 지급
3,여행용 멀티 어댑터 플러그 해외여행 트래블어댑터 콘센트,https://cr.shopping.naver.com/adcr.nhn?x=xhcVC...,"10,900원",파워존 스토어,"종류 : 콘센트형, 전원스위치 : 스위치없음, 특징 : 과부하차단",소식알림 할인쿠폰! 포토리뷰 300포인트 적립!
4,멀티어댑터 여행용 어댑터 GaN PPS USB 5포트 GAN PD 65W 파우치 기본구성,https://cr.shopping.naver.com/adcr.nhn?x=z8fZ7...,"39,900원",Dibea,"종류 : 콘센트형, 코드수 : 1구, 특징 : USB충전포트",
...,...,...,...,...,...,...
115,크리스탈클라우드 5포트 PD 70W GaN 전세계 여행용 멀티어댑터 CC-308,https://cr.shopping.naver.com/adcr.nhn?x=zSEBv...,"59,900원",오늘의집,,첫구매 시 최대 2만원 할인 / 네이버페이 최대 3000P 적립 / 토스페이 최대 ...
116,일본 미국 캐나다 대만 돼지코 변환 플러그 해외용 여행용 어댑터 110V 220V,https://cr.shopping.naver.com/adcr.nhn?x=iCIET...,"3,900원",까만몰,,
117,모락 브리큐 USB 4포트 여행용 멀티 플러그 어댑터 MR-BRIQ01,https://cr.shopping.naver.com/adcr.nhn?x=%2BTc...,"24,900원",morac,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 4구, 특징 : 과부하차단...",
118,가우넷 오호 YM105 USB 4포트 C타입 여행용 어댑터 플러그,https://cr.shopping.naver.com/adcr.nhn?x=ocT1b...,"23,900원",가우넷,"코드수 : 4구, 특징 : USB충전포트",


In [5]:
meta.drop_duplicates(subset= ['prodict_sellers'], inplace=True)

In [4]:
# meta

,product_names,product_urls,product_prices,prodict_sellers,product_detail,product_marketings
0,여행 멀티 플러그 해외 여행용 어댑터 일본 멀티탭 콘센트 멀티 어댑터 유럽 중국,https://cr.shopping.naver.com/adcr.nhn?x=dVYa9...,"2,000원",스카이트립,"종류 : 콘센트형, 전원스위치 : 통합스위치, 코드수 : 1구",
1,감비앤코 여행용 멀티 어댑터 해외여행 멀티탭 돼지코 전세계 멀충 PD C타입,https://cr.shopping.naver.com/adcr.nhn?x=cM9sa...,"32,000원",감비앤코,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 1구, 특징 : 과부하차단...",출시기념 특가 행사와 파우치 증정 이벤트
2,지티스 여행용 멀티 어댑터 해외여행 C타입 초고속 전세계 110V 멀티탭,https://cr.shopping.naver.com/adcr.nhn?x=Kve9j...,"31,800원",지티스몰,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 1구, 특징 : 과부하차단...",알림받기 즉시할인 및 포토영상 리뷰 작성 시 즉시! 1000포인트 지급
3,여행용 멀티 어댑터 플러그 해외여행 트래블어댑터 콘센트,https://cr.shopping.naver.com/adcr.nhn?x=xhcVC...,"10,900원",파워존 스토어,"종류 : 콘센트형, 전원스위치 : 스위치없음, 특징 : 과부하차단",소식알림 할인쿠폰! 포토리뷰 300포인트 적립!
4,멀티어댑터 여행용 어댑터 GaN PPS USB 5포트 GAN PD 65W 파우치 기본구성,https://cr.shopping.naver.com/adcr.nhn?x=z8fZ7...,"39,900원",Dibea,"종류 : 콘센트형, 코드수 : 1구, 특징 : USB충전포트",
...,...,...,...,...,...,...
114,해외 여행용 어댑터 보급형 SL 185A 투어테크,https://cr.shopping.naver.com/adcr.nhn?x=ftYyG...,"7,120원",페이퍼비스타,,
115,크리스탈클라우드 5포트 PD 70W GaN 전세계 여행용 멀티어댑터 CC-308,https://cr.shopping.naver.com/adcr.nhn?x=zSEBv...,"59,900원",오늘의집,,첫구매 시 최대 2만원 할인 / 네이버페이 최대 3000P 적립 / 토스페이 최대 ...
117,모락 브리큐 USB 4포트 여행용 멀티 플러그 어댑터 MR-BRIQ01,https://cr.shopping.naver.com/adcr.nhn?x=%2BTc...,"24,900원",morac,"종류 : 콘센트형, 전원스위치 : 스위치없음, 코드수 : 4구, 특징 : 과부하차단...",
118,가우넷 오호 YM105 USB 4포트 C타입 여행용 어댑터 플러그,https://cr.shopping.naver.com/adcr.nhn?x=ocT1b...,"23,900원",가우넷,"코드수 : 4구, 특징 : USB충전포트",


In [6]:
product_urls = meta['product_urls'].to_list()
product_urls = product_urls[60:]

In [9]:
chrome_driver_path = r'C:/Users/shopperhouse/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe'
service = Service(executable_path=chrome_driver_path)

driver = webdriver.Chrome(service=service, options=options)
driver.implicitly_wait(5)

# URL 리스트
# urls = [
#     'https://brand.naver.com/petzmoa/products/2331258451?n_media=33421&n_query=%EA%B0%95%EC%95%84%EC%A7%80%EB%B0%B0%EB%B3%80%ED%8C%A8%EB%93%9C&n_rank=1&n_ad_group=grp-a001-02-000000015674508&n_ad=nad-a001-02-000000095711529&n_campaign_type=2&n_mall_id=ncp_1nl92b_01&n_mall_pid=2331258451&n_ad_group_type=2&n_match=3',
#     'https://smartstore.naver.com/skytrip/products/2009311639?NaPm=ct%3Dlxsfvitk%7Cci%3Db13535124186ae10422c0cb337a370d0a424f176%7Ctr%3Dslsl%7Csn%3D247953%7Chk%3Ddcdaa000f8cef82f9fac9141a84f200e2d072479'
# ]

urls = product_urls

folder_paths = []

exception_urls = []

# URL에서 데이터 수집
for url in urls:
    driver.get(url)
    actual_url = driver.current_url
    print(actual_url)

    if not (actual_url.startswith('https://brand') or actual_url.startswith('https://smartstore')):
        exception_urls.append(actual_url)
        continue

    # 브랜드명 및 제품명 수집 -> 해당 부분 수정 -> 어떤 element를 뽑을지?
    try:
        try:
            brand_element = driver.find_element(By.CSS_SELECTOR, '#pc-storeNameWidget > div > div > a > img')
            brand = brand_element.get_attribute('alt')
        except Exception as e1:
            try:
                brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > img")
                brand = brand_element.get_attribute('alt')
            except Exception as e2:
                try:
                    brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-storeNameWidget > div > div > a > span")
                    brand = brand_element.text
                except Exception as e3:
                    try:
                        brand_element = driver.find_element(By.CSS_SELECTOR, "#pc-gnbWidget > div > div > div._1G2E7OXbG3 > div._3aNsjos9K5 > h1 > a > span")
                        brand = brand_element.text
                    except Exception as e4:
                        print(f"브랜드명 수집 실패 : {e4}")
                        exception_urls.append(url)
                        continue

        product = driver.find_element(By.CSS_SELECTOR, '#content > div > div._2-I30XS1lA > div._2QCa6wHHPy > fieldset > div._3k440DUKzy > div._1eddO7u4UC > h3').text
    except Exception as e:
        print(f"브랜드명 및 상품명 수집 실패 : {e}")
        exception_urls.append(url)
        continue

    sanitized_product = product.replace("/","_").replace(" ","_")
    folder_name = sanitized_product
    # folder_name = f"{brand}_{product}"
    folder_path = os.path.join("C:/Users/shopperhouse/Desktop/C_Image", folder_name) # local_path
    os.makedirs(folder_path, exist_ok=True)
    folder_paths.append((folder_name, folder_path))

    # 상세페이지 더보기 클릭하려면 마우스를 scroll 해야 함
    scroll_location = driver.execute_script("return document.body.scrollHeight")
    while True:
        # 현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        # 늘어난 스크롤 높이
        scroll_height = driver.execute_script("return document.body.scrollHeight")
        # 늘어난 스크롤 위치와 이동 전 위치 같으면 종료
        if scroll_location == scroll_height:
            break
        # 같지 않으면 스크롤 위치 값을 수정하여 같아질 때까지 반복
        else:
            scroll_location = driver.execute_script("return document.body.scrollHeight")


    count = 0

    try:
        # 상세정보 버튼
        driver.find_element(By.CSS_SELECTOR, '#_productFloatingTab > div > div._27jmWaPaKy._1dDHKD1iiX > ul > li:nth-child(1) > a').click()
        time.sleep(3)

        # 상세정보 펼쳐보기 버튼
        driver.find_element(By.CSS_SELECTOR, '#INTRODUCE > div > div._3osy73V_eD._1Hc_ju_IXp > button').click()
        time.sleep(3)

    except Exception as e:
        print(f"상세정보 버튼 클릭 실패: {e}")
        exception_urls.append(url)
        continue


    # 셀레니움으로 html 가져오기
    html_source = driver.page_source

    # bs4로 html 파싱
    soup = BeautifulSoup(html_source, 'html.parser')
    time.sleep(0.5)

    # 상세정보 전체
    images = soup.findAll('a', {'class': 'se-module-image-link __se_image_link __se_link'})

    # 한 페이지 내에서 수집 가능한 사진 리스트에 저장
    for image in range(len(images)):
        # 4-4. 이미지 수집
        image_tag = images[image].findAll('img', {'class': 'se-image-resource'})
        if image_tag:
            image_link = image_tag[0]['data-src']
            print(f'Review Image Link: {image_link}')  # 이미지 링크 출력
            image_link = image_link.replace('https', 'http')
            opener = urllib.request.build_opener()
            opener.addheaders = [('User-Agent', 'Mozilla/5.0')]
            urllib.request.install_opener(opener)
            local_img_path = os.path.join(folder_path, f'{count}.jpg')
            print(local_img_path)
            urllib.request.urlretrieve(image_link, local_img_path)
            count += 1

            # # s3에도 업로드
            # ecommerce = 'naver'
            # crawling_type = 'image'
            # category = '전기용품'         # 여기부터 변경
            # search_type = 'keyword'
            # file_type = 'jpg'
            # jpg_fullname = f'{folder_name}/{count}'
            # jpg_file_path = local_img_path

            # save_image_jpg_data(ecommerce, crawling_type, category, search_type, file_type, jpg_file_path, jpg_fullname)

print('done')
driver.quit()

for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

     # S3에 업로드
    ecommerce = 'naver'
    crawling_type = 'image'
    category = '전기용품'
    file_type = 'pdf'
    ctg_fullname = folder_name
    search_type = 'keyword'
    save_image_pdf_data(ecommerce, crawling_type, category, file_type, pdf_output_path, ctg_fullname, search_type)

https://cr.shopping.naver.com/adcr.nhn?x=N2nZ0lDnnXgOA1hKhF%2Bcuv%2F%2F%2Fw%3D%3Ds0m0eE0nBgUFtfHmtn%2F4EbKv79NzRBJ6zwzJkeJoTnCKedMUWqnXIzj66FooS5oY%2Fv4xIutM3vXVgI4zEkx8fs0dq4%2BkYuadFRUPdoZpKIvZwZpk8wau1Rv8QkXowOr0BFW7CHaYPRn5CkYTJm%2Fz2lC4NcfoJu3Q2kcTGPaESTD7b99QTm8EZjSvoIg50evgu6UcMFIeGEMsdhCCdbeyPAccsc9undkWY593c1K4sHep6uDrMx0%2B%2BlVrgjHArRouHTx4XXOEjv30cULQLA0lroAWarRACkNu62%2FwsI4Vi7OZr7SMQ%2FCS7rb225LghAQdk99UOAgZWppyGt6unj8r2pqZ5iEZn1tmBIRMMVdSD%2BiiGwfcryKAOtbHpI9JiBgbeXGUdkq06yN1yum8QupjYw2vqR2unsG9Pw6XBns%2FftjJmn%2B6FVZ%2Fm0w7K7lHYQAO7S%2BBQGJfcJe3cZb%2BuigSj7saf%2FtJ78l8Buv34JeKbK5PXu9%2FbTMMj8ShqDHul8W1ptIcs9RytHHkySygdEshlGguriFLgGoZT9zSLkan1B1bIdVfipZ%2F%2FueNs86DRIlyC0N0EQ5TdsbHkDajkk8qJUjM1QyqzEeEtbt0l85qiI86tXX6T2qurhY99CE03P%2Bd6tcIoTb2pcREap04XUEuv04M7PzZNWQrMToSrxBcrndFlNzdJFgkoR1MOL3Zb6EWFuyWG3gsdKZ%2Fd%2B%2BA5Qnx9JH1DXsv6VqOolcZgfvLRWRNXc50eBITwugSNPH3Ok09sVpGw61BSdoQRvEGgfFlc5w%3D%3D&nvMid=41306611031&catId=50003455
https://smartstore.naver.com/banglelife/prod

In [2]:
import os
base_folder_path = 'C:/Users/shopperhouse/Desktop/C_Image'

# 하위 폴더들 탐색
folder_paths = [(folder_name, os.path.join(base_folder_path, folder_name)) 
                for folder_name in os.listdir(base_folder_path) 
                if os.path.isdir(os.path.join(base_folder_path, folder_name))]

In [12]:
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

     # S3에 업로드
    ecommerce = 'naver'
    crawling_type = 'image'
    category = '전기용품'
    file_type = 'pdf'
    ctg_fullname = folder_name
    search_type = 'keyword'
    save_image_pdf_data(ecommerce, crawling_type, category, file_type, pdf_output_path, ctg_fullname, search_type)

PDF saved to C:/Users/shopperhouse/Desktop/C_Image\MOGICS_여행용_멀티아답터_멀티탭_해외_플러그_모직스_파워_슈퍼_베이글\MOGICS_여행용_멀티아답터_멀티탭_해외_플러그_모직스_파워_슈퍼_베이글.pdf
Error uploading PDF to S3: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'MOGICS_여행용_멀티아답터_멀티탭_해외_플러그_모직스_파워_슈퍼_베이글'
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\yullshop_여행용_멀티_어댑터_해외여행_초고속_C타입_충전_멀티탭\yullshop_여행용_멀티_어댑터_해외여행_초고속_C타입_충전_멀티탭.pdf
Error uploading PDF to S3: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'yullshop_여행용_멀티_어댑터_해외여행_초고속_C타입_충전_멀티탭'
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\감비앤코_여행용_멀티_어댑터_해외여행_멀티탭_돼지코_전세계_멀충_PD_C타입\감비앤코_여행용_멀티_어댑터_해외여행_멀티탭_돼지코_전세계_멀충_PD_C타입.pdf
Error uploading PDF to S3: [WinError 2] 지정된 파일을 찾을 수 없습니다: '감비앤코_여행용_멀티_어댑터_해외여행_멀티탭_돼지코_전세계_멀충_PD_C타입'
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\락앤락_여행용_멀티어댑터_5포트_USB_C타입_포함\락앤락_여행용_멀티어댑터_5포트_USB_C타입_포함.pdf
Error uploading PDF to S3: [WinError 2] 지정된 파일을 찾을 수 없습니다: '락앤락_여행용_멀티어댑터_5포트_USB_C타입_포함'
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\러킷_해외_여행용_멀티_어댑터_콘센트\러킷

In [11]:
pdf_output_path

'C:/Users/shopperhouse/Desktop/C_Image\\히위고_여행용_멀티어댑터_4포트_고속_충전_PD_플러그\\히위고_여행용_멀티어댑터_4포트_고속_충전_PD_플러그.pdf'

In [7]:
for folder_name, folder_path in folder_paths:
    # Define the PDF dimensions (A4 size in mm)
    pdf_width = 210
    pdf_height = 297

    # Create a PDF object
    pdf = FPDF()

    # Iterate through the image files in the folder
    for image_file in os.listdir(folder_path):
        if image_file.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder_path, image_file)
            # print(image_path)

            # Open the image file
            image = Image.open(image_path)

            # convert image to rgb
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert image dimensions from pixels to mm (assuming 72 DPI for FPDF)
            width_mm = image.width * 25.4 / 72
            height_mm = image.height * 25.4 / 72

            # Calculate the scaling factor to fit the image within the PDF page
            scaling_factor = min(pdf_width / width_mm, pdf_height / height_mm)

            # Calculate new dimensions while maintaining the aspect ratio
            new_width_mm = width_mm * scaling_factor
            new_height_mm = height_mm * scaling_factor

            # Calculate the position to center the image on the page
            x_position = (pdf_width - new_width_mm) / 2
            y_position = (pdf_height - new_height_mm) / 2

            # Add a new page to the PDF
            pdf.add_page()

            # Insert the image into the PDF at the calculated position and size
            pdf.image(image_path, x_position, y_position, new_width_mm, new_height_mm)

    # Save the PDF to a file
    # Save the PDF to a file
    pdf_output_path = os.path.join(folder_path, f"{folder_name}.pdf")
    pdf.output(pdf_output_path)
    print(f"PDF saved to {pdf_output_path}")

PDF saved to C:/Users/shopperhouse/Desktop/C_Image\MOGICS_여행용_멀티아답터_멀티탭_해외_플러그_모직스_파워_슈퍼_베이글\MOGICS_여행용_멀티아답터_멀티탭_해외_플러그_모직스_파워_슈퍼_베이글.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\yullshop_여행용_멀티_어댑터_해외여행_초고속_C타입_충전_멀티탭\yullshop_여행용_멀티_어댑터_해외여행_초고속_C타입_충전_멀티탭.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\감비앤코_여행용_멀티_어댑터_해외여행_멀티탭_돼지코_전세계_멀충_PD_C타입\감비앤코_여행용_멀티_어댑터_해외여행_멀티탭_돼지코_전세계_멀충_PD_C타입.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\락앤락_여행용_멀티어댑터_5포트_USB_C타입_포함\락앤락_여행용_멀티어댑터_5포트_USB_C타입_포함.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\러킷_해외_여행용_멀티_어댑터_콘센트\러킷_해외_여행용_멀티_어댑터_콘센트.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\멀티어댑터_2포트_해외_여행용_어댑터_USB_멀티플러그_만능어답터\멀티어댑터_2포트_해외_여행용_어댑터_USB_멀티플러그_만능어답터.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\멀티어댑터_여행용_어댑터_GaN_PPS_USB_5포트_GAN_PD_65W_파우치_기본구성\멀티어댑터_여행용_어댑터_GaN_PPS_USB_5포트_GAN_PD_65W_파우치_기본구성.pdf
PDF saved to C:/Users/shopperhouse/Desktop/C_Image\멜리언스_PD35W_5포트_해외여행_초고속충전_멀티어댑터_C타입_돼지코_